# Tutorial

## API

The `gpu_tracker` package provides the `Tracker` class which uses an underlying thread to measure computational resource usage, namely the compute time, maximum RAM used, and maximum GPU RAM used. The `start()` method starts this thread which tracks usage in the background. After calling `start()`, write the code to measure resource usage, followed by calling the `stop()` method. The compute time will be the time from the call to `start()` to the call to `stop()` and the RAM and GPU RAM quantities will be the amount of RAM used by the code that's in between `start()` and `stop()`.

In [1]:
import gpu_tracker as gput

In [2]:
tracker = gput.Tracker()
tracker.start()
# Perform expensive operations
tracker.stop()

The `Tracker` class implements the `__str__` method so it can be printed as a string with the values and units of each computational resource formatted.

In [3]:
print(tracker)

Max RAM:
   Unit: gigabytes
   System capacity: 67.254
   System: 4.417
   Main:
      Total RSS: 0.061
      Private RSS: 0.05
      Shared RSS: 0.011
   Descendents:
      Total RSS: 0.0
      Private RSS: 0.0
      Shared RSS: 0.0
   Combined:
      Total RSS: 0.063
      Private RSS: 0.052
      Shared RSS: 0.011
Max GPU RAM:
   Unit: gigabytes
   Main: 0.0
   Descendents: 0.0
   Combined: 0.0
Compute time:
   Unit: hours
   Time: 0.0


The system capacity is a constant for the total RAM capacity across the entire operating system, not to be confused with the maximum system RAM which is the maximum OS RAM that was actually used over the duration of the computational-resource tracking. Both the RAM and GPU RAM are split up into 3 sections, namely the usage of the main process itself followed by the summed usage of any descendent processes it may have (i.e. child processes, grandchild processes, etc.), and combined usage which is the sum of the main and its descendent processes. RAM is divided further to include the private RSS (RAM usage unique to the process), shared RSS (RAM that's shared by a process and at least one other process), and total RSS (the sum of private and shared RSS). The private and shared RSS values are only available on Linux distributions. So for non-linux operating systems, the privated and shared RSS will remain 0 and only the total RSS will be reported. Theoretically, the combined total RSS would never exceed the overall system RAM usage, but inaccuracies resulting from shared RSS can cause this to happen, especially for non-linux operating systems (see note below).

The `Tracker` assumes that GPU memory is not shared accross multiple processes and if it is, the reported GPU RAM of "descendent" and "combined" may be an overestimation.

The compute time is the real time that the computational-resource tracking lasted (as compared to CPU time).

***NOTE** The keywords "descendents" and "combined" in the output above indicate a sum of the RSS used by multiple processes. It's important to keep in mind that on non-linux operating systems, this sum does not take into account shared memory but rather adds up the total RSS of all processes, which can lead to an overestimation. For Linux distributions, however, pieces of shared memory are only counted once.*

The `Tracker` can alternatively be used as a context manager rather than explicitly calling `start()` and `stop()`.

In [4]:
with gput.Tracker() as tracker:
    # Perform expensive operations
    pass
print(tracker)

Max RAM:
   Unit: gigabytes
   System capacity: 67.254
   System: 4.398
   Main:
      Total RSS: 0.063
      Private RSS: 0.052
      Shared RSS: 0.011
   Descendents:
      Total RSS: 0.0
      Private RSS: 0.0
      Shared RSS: 0.0
   Combined:
      Total RSS: 0.063
      Private RSS: 0.052
      Shared RSS: 0.011
Max GPU RAM:
   Unit: gigabytes
   Main: 0.0
   Descendents: 0.0
   Combined: 0.0
Compute time:
   Unit: hours
   Time: 0.0


The units of the computational resources can be modified as desired. For example, to measure the RAM in megabytes, the GPU RAM in kilobytes, and the compute time in seconds:

In [5]:
with gput.Tracker(ram_unit='megabytes', gpu_ram_unit='kilobytes', time_unit='seconds') as tracker:
    # Perform expensive operations
    pass
print(tracker)

Max RAM:
   Unit: megabytes
   System capacity: 67254.161
   System: 4378.161
   Main:
      Total RSS: 63.336
      Private RSS: 52.171
      Shared RSS: 11.166
   Descendents:
      Total RSS: 0.0
      Private RSS: 0.0
      Shared RSS: 0.0
   Combined:
      Total RSS: 63.341
      Private RSS: 52.175
      Shared RSS: 11.166
Max GPU RAM:
   Unit: kilobytes
   Main: 0.0
   Descendents: 0.0
   Combined: 0.0
Compute time:
   Unit: seconds
   Time: 0.063


The same information as the text format can be provided as a dictionary via the `to_json()` method of the `Tracker`.

In [6]:
import json
print(json.dumps(tracker.to_json(), indent=1))

{
 "max_ram": {
  "unit": "megabytes",
  "system_capacity": 67254.161408,
  "system": 4378.161152,
  "main": {
   "total_rss": 63.336448,
   "private_rss": 52.170752,
   "shared_rss": 11.165695999999999
  },
  "descendents": {
   "total_rss": 0.0,
   "private_rss": 0.0,
   "shared_rss": 0.0
  },
  "combined": {
   "total_rss": 63.340543999999994,
   "private_rss": 52.174848,
   "shared_rss": 11.165695999999999
  }
 },
 "max_gpu_ram": {
  "unit": "kilobytes",
  "main": 0.0,
  "descendents": 0.0,
  "combined": 0.0
 },
 "compute_time": {
  "unit": "seconds",
  "time": 0.06275105476379395
 }
}


The `Tracker` class additionally has fields that provide the usage information for each computational resource as python data classes. 

In [7]:
tracker.max_ram

MaxRAM(unit='megabytes', system_capacity=67254.161408, system=4378.161152, main=RSSValues(total_rss=63.336448, private_rss=52.170752, shared_rss=11.165695999999999), descendents=RSSValues(total_rss=0.0, private_rss=0.0, shared_rss=0.0), combined=RSSValues(total_rss=63.340543999999994, private_rss=52.174848, shared_rss=11.165695999999999))

In [8]:
tracker.max_ram.unit

'megabytes'

In [9]:
tracker.max_ram.main

RSSValues(total_rss=63.336448, private_rss=52.170752, shared_rss=11.165695999999999)

In [10]:
tracker.max_ram.main.total_rss

63.336448

In [11]:
tracker.max_gpu_ram

MaxGPURAM(unit='kilobytes', main=0.0, descendents=0.0, combined=0.0)

In [12]:
tracker.compute_time

ComputeTime(unit='seconds', time=0.06275105476379395)

## CLI

The `gpu-tracker` package also comes with a commandline interface that can track the computational-resource-usage of any shell command, not just python code. Entering `gpu-tracker -h` in a shell will show the help message.

In [13]:
!gpu-tracker -h

Tracks the computational resource usage (RAM, GPU RAM, and compute time) of a process corresponding to a given shell command.

Usage:
    gpu-tracker --execute=<command> [--output=<output>] [--format=<format>] [--st=<sleep-time>] [--ru=<ram-unit>] [--gru=<gpu-ram-unit>] [--tu=<time-unit>]

Options:
    -h --help               Show this help message.
    -e --execute=<command>  The command to run along with its arguments all within quotes e.g. "ls -l -a".
    -o --output=<output>    File path to store the computational-resource-usage measurements. If not set, prints measurements to the screen.
    -f --format=<format>    File format of the output. Either 'json' or 'text'. Defaults to 'text'.
    --st=<sleep-time>       The number of seconds to sleep in between usage-collection iterations.
    --ru=<ram-unit>         One of 'bytes', 'kilobytes', 'megabytes', 'gigabytes', or 'terabytes'.
    --gru=<gpu-ram-unit>    One of 'bytes', 'kilobytes', 'megabytes', 'gigabytes', or 'terabytes'.
   

The `-e` or `--execute` is a required option where the desired shell command is provided, with both the command and its proceeding arguments surrounded by quotes. Below is an example of running the `sleep` command with an argument of 2 seconds. When the command completes, its status code is reported. 

In [14]:
!gpu-tracker -e 'sleep 2'

Resource tracking complete. Process completed with status code: 0
Max RAM:
   Unit: gigabytes
   System capacity: 67.254
   System: 4.398
   Main:
      Total RSS: 0.002
      Private RSS: 0.0
      Shared RSS: 0.002
   Descendents:
      Total RSS: 0.0
      Private RSS: 0.0
      Shared RSS: 0.0
   Combined:
      Total RSS: 0.002
      Private RSS: 0.0
      Shared RSS: 0.002
Max GPU RAM:
   Unit: gigabytes
   Main: 0.0
   Descendents: 0.0
   Combined: 0.0
Compute time:
   Unit: hours
   Time: 0.0


The units of the computational resources can be modified. For example, --tu stands for time-unit and --ru stands for ram-unit.

In [15]:
!gpu-tracker -e 'sleep 2' --tu=seconds --ru=megabytes

Resource tracking complete. Process completed with status code: 0
Max RAM:
   Unit: megabytes
   System capacity: 67254.161
   System: 4425.966
   Main:
      Total RSS: 1.663
      Private RSS: 0.139
      Shared RSS: 1.524
   Descendents:
      Total RSS: 0.0
      Private RSS: 0.0
      Shared RSS: 0.0
   Combined:
      Total RSS: 1.663
      Private RSS: 0.139
      Shared RSS: 1.524
Max GPU RAM:
   Unit: gigabytes
   Main: 0.0
   Descendents: 0.0
   Combined: 0.0
Compute time:
   Unit: seconds
   Time: 1.075


By default, the computational-resource-usage statistics are printed to the screen. The `-o` or `--output` option can be specified to store that same content in a file.

In [16]:
!gpu-tracker -e 'sleep 2' -o out.txt 

Resource tracking complete. Process completed with status code: 0


In [17]:
!cat out.txt

Max RAM:
   Unit: gigabytes
   System capacity: 67.254
   System: 4.414
   Main:
      Total RSS: 0.002
      Private RSS: 0.0
      Shared RSS: 0.001
   Descendents:
      Total RSS: 0.0
      Private RSS: 0.0
      Shared RSS: 0.0
   Combined:
      Total RSS: 0.002
      Private RSS: 0.0
      Shared RSS: 0.001
Max GPU RAM:
   Unit: gigabytes
   Main: 0.0
   Descendents: 0.0
   Combined: 0.0
Compute time:
   Unit: hours
   Time: 0.0

By default, the format of the output is "text". The `-f` or `--format` option can specify the format to be "json" instead. 

In [18]:
!gpu-tracker -e 'sleep 2' -f json

Resource tracking complete. Process completed with status code: 0
{
 "max_ram": {
  "unit": "gigabytes",
  "system_capacity": 67.254161408,
  "system": 4.414156800000001,
  "main": {
   "total_rss": 0.0015892480000000001,
   "private_rss": 0.00013516800000000002,
   "shared_rss": 0.0014540800000000002
  },
  "descendents": {
   "total_rss": 0.0,
   "private_rss": 0.0,
   "shared_rss": 0.0
  },
  "combined": {
   "total_rss": 0.0015892480000000001,
   "private_rss": 0.00013516800000000002,
   "shared_rss": 0.0014540800000000002
  }
 },
 "max_gpu_ram": {
  "unit": "gigabytes",
  "main": 0.0,
  "descendents": 0.0,
  "combined": 0.0
 },
 "compute_time": {
  "unit": "hours",
  "time": 0.00029873490333557127
 }
}


In [19]:
!gpu-tracker -e 'sleep 2' -f json -o out.json

Resource tracking complete. Process completed with status code: 0


In [20]:
!cat out.json

{
 "max_ram": {
  "unit": "gigabytes",
  "system_capacity": 67.254161408,
  "system": 4.407083008,
  "main": {
   "total_rss": 0.001830912,
   "private_rss": 0.00013516800000000002,
   "shared_rss": 0.0016957440000000001
  },
  "descendents": {
   "total_rss": 0.0,
   "private_rss": 0.0,
   "shared_rss": 0.0
  },
  "combined": {
   "total_rss": 0.001830912,
   "private_rss": 0.00013516800000000002,
   "shared_rss": 0.0016957440000000001
  }
 },
 "max_gpu_ram": {
  "unit": "gigabytes",
  "main": 0.0,
  "descendents": 0.0,
  "combined": 0.0
 },
 "compute_time": {
  "unit": "hours",
  "time": 0.0002994798951678806
 }
}